In [2]:
import pandas as pd
df=pd.read_csv("../data/processed/final_merged_file.csv")

In [3]:
df['match_date'] = pd.to_datetime(df['match_date'])

calculationg total match per goals for home team

In [4]:
home_df = df[[
    'match_date', 'season', 'league_id',
    'home_team_api_id', 'home_team_goal', 'away_team_goal',"overall_rating","sprint_speed","reactions","strength"
]].rename(columns={
    'home_team_api_id': 'team_id',
    'home_team_goal': 'goals_for',
    'away_team_goal': 'goals_against'
})

for away team

In [5]:
away_df = df[[
    'match_date', 'season', 'league_id',
    'away_team_api_id', 'away_team_goal', 'home_team_goal',"overall_rating","sprint_speed","reactions","strength"
]].rename(columns={
    'away_team_api_id': 'team_id',
    'away_team_goal': 'goals_for',
    'home_team_goal': 'goals_against'
})

In [6]:
team_matches = pd.concat([home_df, away_df], ignore_index=True)

In [7]:
team_matches = team_matches.sort_values(
    ['team_id', 'season', 'match_date']).reset_index(drop=True)

In [8]:
team_matches['rest_days'] = (
    team_matches
    .groupby(['team_id', 'season'])['match_date']
    .diff()
    .dt.days
)

In [9]:
def workload_category(x):
    if pd.isna(x):
        return None
    elif x <= 3:
        return 'High congestion'
    elif x <= 6:
        return 'Medium congestion'
    else:
        return 'Normal rest'

team_matches['workload_category'] = team_matches['rest_days'].apply(workload_category)

In [10]:
team_matches['goal_diff'] = (
    team_matches['goals_for'] - team_matches['goals_against']
)
final_table = team_matches.dropna(subset=['rest_days']).reset_index(drop=True)
final_table.columns

Index(['match_date', 'season', 'league_id', 'team_id', 'goals_for',
       'goals_against', 'overall_rating', 'sprint_speed', 'reactions',
       'strength', 'rest_days', 'workload_category', 'goal_diff'],
      dtype='object')

In [11]:
# Does playing multiple matches in a short time window reduce overall_rating?
final_table[["rest_days","overall_rating"]].corr()

,rest_days,overall_rating
rest_days,1.00000,-0.01934
overall_rating,-0.01934,1.00000


In [ ]:
#How does short‑term match congestion (rest days) influence player performance metrics such as overall_rating, sprint_speed, and reactions?
final_table[["rest_days","overall_rating","sprint_speed","reactions"]].corr()

,rest_days,overall_rating,sprint_speed,reactions
rest_days,1.000000,-0.019340,-0.050132,-0.015242
overall_rating,-0.019340,1.000000,0.254734,0.769224
sprint_speed,-0.050132,0.254734,1.000000,0.273647
reactions,-0.015242,0.769224,0.273647,1.000000


In [ ]:
#final_table.to_csv("team_match_workload_performance.csv",index=False)